# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


# Initialization of SparkSession

In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName="jupyter")

from pyspark.sql import SparkSession, Row
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-12 08:54:53,129 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
!hdfs dfs -ls hdfs://localhost:9000/user/jovyan/

Found 2 items
drwxr-xr-x   - jovyan supergroup          0 2024-01-12 08:54 hdfs://localhost:9000/user/jovyan/.sparkStaging
-rw-r--r--   1 jovyan supergroup   32241574 2024-01-08 11:09 hdfs://localhost:9000/user/jovyan/clickstream.csv


In [3]:
!hdfs dfs -put ./clickstream.csv hdfs://localhost:9000/user/jovyan/clickstream.csv

put: `hdfs://localhost:9000/user/jovyan/clickstream.csv': File exists


# Spark SQL solution

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [5]:
# Load the data into a DataFrame
clickstream = spark.read.option("header", True).option("delimiter", "\t").csv("clickstream.csv")

In [6]:
# Create a temporary view to query with SQL
clickstream.createOrReplaceTempView("clickstream")

In [7]:
# Perform the query
result = spark.sql('''
SELECT route, count(route) as cnt 
FROM (
    SELECT user_id, session_id, CONCAT_WS("-", COLLECT_LIST(event_page)) as route 
    FROM (
        SELECT user_id, session_id, event_page,
            LAG(event_page) OVER (
                PARTITION BY user_id, session_id 
                ORDER BY timestamp
            ) as prev_page 
        FROM (
            SELECT user_id, session_id, event_type, event_page, timestamp,
                MAX(CASE WHEN event_type LIKE '%error%' THEN 1 ELSE 0 END) OVER (
                    PARTITION BY user_id, session_id 
                    ORDER BY timestamp
                ) as error
            FROM clickstream
            DISTRIBUTE BY user_id, session_id
            SORT BY user_id, session_id, timestamp
        ) 
        WHERE event_type = 'page' AND error = 0
    ) 
    WHERE event_page != prev_page OR prev_page IS NULL
    GROUP BY user_id, session_id
) 
GROUP BY route
ORDER BY cnt DESC
LIMIT 30
''')

In [8]:
# Save the result to a CSV file
result.toPandas().to_csv('SQLResult.csv', header=False, index=False, sep='\t')

In [9]:
! cat SQLResult.csv

main	8184
main-archive	1113
main-rabota	1046
main-internet	897
main-bonus	869
main-news	769
main-tariffs	677
main-online	587
main-vklad	518
main-rabota-archive	170
main-archive-rabota	167
main-bonus-archive	143
main-rabota-bonus	139
main-news-rabota	135
main-bonus-rabota	135
main-archive-internet	132
main-rabota-news	130
main-internet-rabota	129
main-archive-news	126
main-rabota-internet	124
main-internet-archive	123
main-archive-bonus	117
main-internet-bonus	115
main-tariffs-internet	114
main-news-archive	113
main-news-internet	109
main-archive-tariffs	104
main-internet-news	103
main-tariffs-archive	103
main-rabota-main	94


# Spark RDD solution

In [10]:
# Load data as an RDD
clickstream = sc.textFile("clickstream.csv")

In [11]:
# Remove the header
header = clickstream.first()
clickstream = clickstream.filter(lambda line: line != header)

In [12]:
# Process clickstream
clickstream = clickstream.map(lambda line: line.split('\t')) \
                         .map(lambda cols: (cols[0] + '-' + cols[1], cols[2], cols[3], cols[4]))

In [13]:
# Get minimum error times
min_error_time = clickstream.filter(lambda x: 'error' in x[1]) \
                            .map(lambda x: (x[0], int(x[3]))) \
                            .reduceByKey(min)

In [14]:
# Join clickstream with minimum error times, then filter before events
clickstream_joined = clickstream.map(lambda x: (x[0], x)) \
                                .leftOuterJoin(min_error_time) \
                                .filter(lambda x: 'error' not in x[1][0][1] and int(x[1][0][3]) < (x[1][1] if x[1][1] is not None else float('inf'))) \
                                .map(lambda x: x[1][0])

In [15]:
# Process clickstream
from itertools import groupby

clickstream_grouped = clickstream_joined.map(lambda x: (x[0], (x[2], x[3]))) \
                                        .groupByKey() \
                                        .mapValues(lambda values: sorted(values, key=lambda x: x[1]))

In [16]:
# Helper function
def get_route(row):
    return '-'.join([key for key, _ in groupby((x[0] for x in row))])

In [17]:
# Create routs
route_counts = clickstream_grouped.map(lambda x: (get_route(x[1]), 1)) \
                                  .reduceByKey(lambda a, b: a + b) \
                                  .sortBy(lambda x: x[1], ascending=False) \
                                  .take(30)

In [18]:
# Print results
for tup in route_counts:
    print(tup)

('main', 8184)
('main-archive', 1113)
('main-rabota', 1047)
('main-internet', 897)
('main-bonus', 870)
('main-news', 769)
('main-tariffs', 677)
('main-online', 587)
('main-vklad', 518)
('main-rabota-archive', 170)
('main-archive-rabota', 167)
('main-bonus-archive', 143)
('main-rabota-bonus', 139)
('main-news-rabota', 135)
('main-bonus-rabota', 135)
('main-archive-internet', 132)
('main-rabota-news', 130)
('main-internet-rabota', 129)
('main-archive-news', 126)
('main-rabota-internet', 124)
('main-internet-archive', 123)
('main-archive-bonus', 117)
('main-internet-bonus', 115)
('main-tariffs-internet', 114)
('main-news-archive', 113)
('main-news-internet', 109)
('main-archive-tariffs', 104)
('main-internet-news', 103)
('main-tariffs-archive', 103)
('main-rabota-main', 94)


In [19]:
import csv

with open('RDDResult.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['route', 'count'])

    for data in route_counts:
        writer.writerow(data)

# Spark DF solution

In [20]:
from pyspark.sql.functions import col, expr, lag, when, collect_list, concat_ws
from pyspark.sql.window import Window

In [21]:
# Read data
clickstream = (
    spark.read.options(header=True, delimiter="\t", inferSchema=True).csv("clickstream.csv")
    .withColumn("error_time", when(col("event_type").rlike("error"), col("timestamp")).otherwise(float('inf')))
)

In [22]:
# Find minimum error time if any for each (session_id, user_id) pair
min_error_time = clickstream.groupBy("session_id", "user_id").min("error_time").alias("min_error_time")

In [23]:
# Join clickstream with minimum error time and adjust the 'timestamp' column on errors
clickstream_adjusted = (
    clickstream.join(min_error_time, ["session_id", "user_id"], how="left")
    .withColumn("timestamp", when(col("event_type").rlike("error"), col("timestamp") + 1).otherwise(col("timestamp")))
    .where(col("timestamp") <= col("min(error_time)"))
    .where(col("event_type") != "event")
)

In [24]:
# Calculate the lag of the 'event_page' column per session_id, user_id, and order by timestamp
partition_window = Window.partitionBy("session_id", "user_id").orderBy("timestamp")
clickstream_with_lag = clickstream_adjusted.withColumn("LAG", lag("event_page", 1).over(partition_window))

In [25]:
# Filter out records where the current event_page is the same as the previous (LAG column)
distinct_clickstream = clickstream_with_lag.filter(expr("event_page IS DISTINCT FROM LAG"))

In [26]:
# Aggregate the distinct event_page values to form the route for each session_id, user_id pair
routes = (
    distinct_clickstream.groupBy("session_id", "user_id")
    .agg(concat_ws("-", collect_list("event_page")).alias("route"))
)

In [27]:
# Count occurrences of each route, sort by count, and take the top 30
top_routes = (
    routes.groupBy("route").count()
    .orderBy("count", ascending=False)
    .limit(30)
)

In [28]:
top_routes.show(30) # truncate=False)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1112|
|         main-rabota| 1048|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  768|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  517|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  138|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  118|
| main-internet-bonus|  115|
|main-tariffs-inte...|  113|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+



In [29]:
top_routes.toPandas().to_csv('DataFrameResult.csv', header=False, index=False, sep='\t')

## Stop session

In [30]:
spark.stop()